# Lab 2 - Data Collection & Pre-Processing

## 1. Hello, Data!

In [89]:
import pandas as pd
import numpy as np

# Keep first 500
# data = pd.read_csv('1000 Sales Records.csv')
# df_500 = data.head(500)
# df_500.to_csv('500_Sales_Records.csv', index=False)

data_org = pd.read_csv('500_Sales_Records.csv')
data_org.head(3)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,10/18/2014,686800706,10/31/2014,8446,437.20,263.33,3692591.20,2224085.18,1468506.02
1,North America,Canada,Vegetables,Online,M,11/7/2011,185941302,12/8/2011,3018,154.06,90.93,464953.08,274426.74,190526.34
2,Middle East and North Africa,Libya,Baby Food,Offline,C,10/31/2016,246222341,12/9/2016,1517,255.28,159.42,387259.76,241840.14,145419.62


## 8. Transformations — Rename (Item Type -> product, Unit Price -> price, Units Sold -> quantity, Order Date -> date)

In [90]:
"""
Cell generated by Data Wrangler.
"""
from datetime import datetime

def clean_data(data):
    # Derive column 'date' from column: 'Order Date'
    def date(Order_Date):
        """
        Transform based on the following examples:
           Order Date      Output
        1: "10/18/2014" => "2014-10-18"
        2: "11/7/2011"  => "2011-11-07"
        """
        date1 = datetime.strptime(Order_Date, "%m/%d/%Y")
        return date1.strftime("%Y-%m-%d")
    data.insert(6, "date", data.apply(lambda row : date(row["Order Date"]), axis=1))
    # Derive column 'product' from column: 'Item Type'
    # Transform based on the following examples:
    #    Item Type      Output
    # 1: "Cosmetics" => "Cosmetics"
    data.insert(3, "product", data["Item Type"])
    # Derive column 'price' from column: 'Unit Price'
    # Transform based on the following examples:
    #    Unit Price    Output
    # 1:      437.2 => "437.2"
    data.insert(12, "price", data.apply(lambda row : f'{row["Unit Price"]:03.1f}', axis=1))
    # Derive column 'quantity' from column: 'Units Sold'
    # Transform based on the following examples:
    #    Units Sold    Output
    # 1:      8446L => "8446"
    data.insert(11, "quantity", data.apply(lambda row : f'{row["Units Sold"]:01.0f}', axis=1))
    return data

data_rename = clean_data(data_org.copy())
data_rename.head()

,Region,Country,Item Type,product,Sales Channel,Order Priority,Order Date,date,Order ID,Ship Date,Units Sold,quantity,Unit Price,price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Libya,Cosmetics,Cosmetics,Offline,M,10/18/2014,2014-10-18,686800706,10/31/2014,8446,8446,437.20,437.2,263.33,3692591.20,2224085.18,1468506.02
1,North America,Canada,Vegetables,Vegetables,Online,M,11/7/2011,2011-11-07,185941302,12/8/2011,3018,3018,154.06,154.1,90.93,464953.08,274426.74,190526.34
2,Middle East and North Africa,Libya,Baby Food,Baby Food,Offline,C,10/31/2016,2016-10-31,246222341,12/9/2016,1517,1517,255.28,255.3,159.42,387259.76,241840.14,145419.62
3,Asia,Japan,Cereal,Cereal,Offline,C,4/10/2010,2010-04-10,161442649,5/12/2010,3322,3322,205.70,205.7,117.11,683335.40,389039.42,294295.98
4,Sub-Saharan Africa,Chad,Fruits,Fruits,Offline,H,8/16/2011,2011-08-16,645713555,8/31/2011,9845,9845,9.33,9.3,6.92,91853.85,68127.40,23726.45


## 2. Pick the Right Container

- For row‑wise transforms and analytics, I'll use **pandas.DataFrame**.
- For fast membership/unique checks, I'll use **set**.
- For keyed lookups (e.g., coupons, city mapping), I'll use **dict** / small **DataFrame**.

## 3. Implement Functions and  Data structure

In [91]:
from dataclasses import dataclass

@dataclass
class Cleaner:
    df: pd.DataFrame
    def clean(self) -> pd.DataFrame:
        w = self.df.copy()
        for c in w.select_dtypes(include=['object']).columns:
            w[c] = w[c].astype(str).str.strip()
        if 'Units Sold' in w.columns: w = w[w['Units Sold']>=0]
        if 'Unit Price' in w.columns: w = w[w['Unit Price']>=0]
        if 'Order Date' in w.columns: w['Order Date'] = pd.to_datetime(w['Order Date'], errors='coerce')
        if 'Ship Date' in w.columns:  w['Ship Date']  = pd.to_datetime(w['Ship Date'],  errors='coerce')
        self.df = w; return w
    def total(self, price_col='Unit Price', qty_col='Units Sold') -> float:
        return float((self.df[price_col]*self.df[qty_col]).sum())

cleaner = Cleaner(data_rename)
data_clean = cleaner.clean()
{"rows_before": len(data_rename), "rows_after": len(data_clean), "sold_total": cleaner.total()}

{'rows_before': 500, 'rows_after': 500, 'sold_total': 711192227.21}

## 4. Bulk Loaded — Map DataFrame → dictionaries

In [92]:
# Example: product → average unit price
prod_to_avg_price = (
    data_clean.groupby('Item Type')['Unit Price'].mean().to_dict()
)
list(prod_to_avg_price.items())[:5]

[('Baby Food', 255.27999999999997),
 ('Beverages', 47.45),
 ('Cereal', 205.7),
 ('Clothes', 109.28),
 ('Cosmetics', 437.2)]

## 5. Quick Profiling


In [93]:

price_min = round(float(data_clean['Unit Price'].min()), 2)
price_mean = round(float(data_clean['Unit Price'].mean()), 2)
price_max = round(float(data_clean['Unit Price'].max()), 2)
unique_countries = len(set(data_clean['Country']))
{'min_price': price_min, 'mean_price': price_mean, 'max_price': price_max, 'unique_country_count': unique_countries}

{'min_price': 9.33,
 'mean_price': 274.3,
 'max_price': 668.27,
 'unique_country_count': 171}

## 6. Spot the Grime

In [94]:
issues = {
    "duplicate_order_ids": int(data_clean["Order ID"].duplicated().sum()),
    "null_order_dates": int(data_clean["Order Date"].isna().sum()),
    "negative_units_sold": int((data_clean["Units Sold"] < 0).sum())
}
issues

{'duplicate_order_ids': 0, 'null_order_dates': 0, 'negative_units_sold': 0}

## 7. Cleaning Rules (Before/After)

In [95]:
rules_report = {
    "nulls_before": int(data_rename.isna().sum().sum()),
    "nulls_after": int(data_clean.isna().sum().sum()),
    "duplicates_before": int(data_rename.duplicated().sum()),
    "duplicates_after": int(data_clean.duplicated().sum()),
    "neg_units_before": int((data_rename["Units Sold"] < 0).sum()),
    "neg_units_after": int((data_clean["Units Sold"] < 0).sum())
}
rules_report

{'nulls_before': 0,
 'nulls_after': 0,
 'duplicates_before': 0,
 'duplicates_after': 0,
 'neg_units_before': 0,
 'neg_units_after': 0}

## 8. Transformations — Coupons

In [96]:
# 1) Load and normalize the coupon metadata
coupon_df = pd.read_csv("Secondary_Metadata_File/Discount_Coupon.csv").copy()
coupon_df = coupon_df.rename(columns={
    "Product_Category": "product",
    "Coupon_Code": "coupon_code",
    "Discount_pct": "discount_pct",
    "Month": "Month",
})

# 2) Strip whitespace and clean string columns
for c in ["product", "coupon_code", "Month"]:
    if c in coupon_df.columns:
        coupon_df[c] = coupon_df[c].astype(str).str.strip()

# 3) Convert discount to numeric and normalize to 0–1
coupon_df["discount_pct"] = pd.to_numeric(coupon_df["discount_pct"], errors="coerce").fillna(0.0)
coupon_df.loc[coupon_df["discount_pct"] > 1, "discount_pct"] /= 100.0
coupon_df["discount_pct"] = coupon_df["discount_pct"].clip(0, 1)

# --- NEW: normalize Month names to 3-letter abbreviations (Jan..Dec) ---
month_map = {
    "january":"Jan","jan":"Jan",
    "february":"Feb","feb":"Feb",
    "march":"Mar","mar":"Mar",
    "april":"Apr","apr":"Apr",
    "may":"May",
    "june":"Jun","jun":"Jun",
    "july":"Jul","jul":"Jul",
    "august":"Aug","aug":"Aug",
    "september":"Sep","sept":"Sep","sep":"Sep",
    "october":"Oct","oct":"Oct",
    "november":"Nov","nov":"Nov",
    "december":"Dec","dec":"Dec",
}
coupon_df["Month"] = coupon_df["Month"].astype(str).str.strip()
coupon_df["Month"] = coupon_df["Month"].str.lower().map(month_map).fillna(coupon_df["Month"])

# 4) Keep only one row per Month (highest discount), then ensure all 12 months exist
#    4a. select the row with max discount per month
top_per_month = (
    coupon_df.sort_values(["Month", "discount_pct"], ascending=[True, False])
             .drop_duplicates(subset=["Month"], keep="first")
             .loc[:, ["Month", "coupon_code", "discount_pct"]]
)

#    4b. create a complete Month frame (Jan..Dec)
all_months = pd.DataFrame({"Month": ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]})

#    4c. left-join to force all 12 months to appear, fill defaults for missing months
coupon_by_month = (
    all_months.merge(top_per_month, on="Month", how="left")
              .assign(
                  coupon_code=lambda d: d["coupon_code"].fillna("NONE"),
                  discount_pct=lambda d: d["discount_pct"].fillna(0.0)
              )
)


w = data_clean.copy()
w["date"] = pd.to_datetime(w["date"], errors="coerce")
w["Month"] = w["date"].dt.strftime("%b")
w.loc[w["date"].isna(), "Month"] = "UNK"   # Mark invalid dates as UNK

# 6) Merge sales with monthly coupon info (now guaranteed to have all 12 months)
w = w.merge(coupon_by_month, on="Month", how="left")

# 7) Fill missing values
w["coupon_code"]  = w["coupon_code"].fillna("NONE")
w["discount_pct"] = w["discount_pct"].fillna(0.0)

# 8) Quick check
print("Coupon merge by month completed.")
display(w[["product","date","Month","coupon_code","discount_pct"]].head(10))

print("Monthly coupon table sample:")
display(coupon_by_month.head(12))

Coupon merge by month completed.


/var/folders/my/jx5vmkcx0xsgknytb6g77_bc0000gn/T/ipykernel_76215/3163950902.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3
 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0.3 0.1 0.2 0

,product,date,Month,coupon_code,discount_pct
0,Cosmetics,2014-10-18,Oct,SALE10,0.1
1,Vegetables,2011-11-07,Nov,SALE20,0.2
2,Baby Food,2016-10-31,Oct,SALE10,0.1
3,Cereal,2010-04-10,Apr,SALE10,0.1
4,Fruits,2011-08-16,Aug,SALE20,0.2
5,Cereal,2014-11-24,Nov,SALE20,0.2
6,Cereal,2015-03-04,Mar,SALE30,0.3
7,Clothes,2012-05-17,May,SALE20,0.2
8,Vegetables,2015-01-29,Jan,SALE10,0.1
9,Vegetables,2013-12-24,Dec,SALE30,0.3


Monthly coupon table sample:


,Month,coupon_code,discount_pct
0,Jan,SALE10,0.1
1,Feb,SALE20,0.2
2,Mar,SALE30,0.3
3,Apr,SALE10,0.1
4,May,SALE20,0.2
5,Jun,SALE30,0.3
6,Jul,SALE10,0.1
7,Aug,SALE20,0.2
8,Sep,SALE30,0.3
9,Oct,SALE10,0.1


## 9. Feature Engineering — Customer_ID, shipping_city, days

In [97]:
# Start directly from w (already has date, product, price, quantity, coupon_code, Month)
df = w.copy()

# 1) Create customer_id
order_id_num = pd.to_numeric(df["Order ID"], errors="coerce")
df["customer_id"] = "C" + (order_id_num % 100000).fillna(0).astype(int).astype(str).str.zfill(5)

# 2) Map Country -> shipping_city
city_lu = pd.read_csv("Secondary_Metadata_File/country_city_lookup.csv").copy()
city_lu = city_lu.rename(columns={"Shipping City": "shipping_city"})
city_lu["Country"] = city_lu["Country"].astype(str).str.strip()
city_lu["shipping_city"] = city_lu["shipping_city"].astype(str).str.strip()

df["Country"] = df["Country"].astype(str).str.strip()
df = df.merge(city_lu[["Country", "shipping_city"]], on="Country", how="left")

# fallback if shipping_city missing
df["shipping_city"] = np.where(
    df["shipping_city"].notna() & (df["shipping_city"].str.len() > 0),
    df["shipping_city"],
    df["Country"]
)

# 3) Add days_since_purchase
df["date"] = pd.to_datetime(df["date"], errors="coerce")
valid_dates = df["date"].dropna()
if not valid_dates.empty:
    ref_date = valid_dates.max()
    df["days_since_purchase"] = (ref_date - df["date"]).dt.days
else:
    df["days_since_purchase"] = np.nan

# Keep only the final enriched dataset in one variable
final_data = df.copy()
del df, w

# 4) Final required fields
required_cols = ["date","customer_id","product","price","quantity","coupon_code","shipping_city"]
display(final_data[required_cols].head(10))


,date,customer_id,product,price,quantity,coupon_code,shipping_city
0,2014-10-18,C00706,Cosmetics,437.2,8446,SALE10,Tripoli
1,2011-11-07,C41302,Vegetables,154.1,3018,SALE20,Ottawa
2,2016-10-31,C22341,Baby Food,255.3,1517,SALE10,Tripoli
3,2010-04-10,C42649,Cereal,205.7,3322,SALE10,Tokyo
4,2011-08-16,C13555,Fruits,9.3,9845,SALE20,Chad
5,2014-11-24,C58888,Cereal,205.7,9528,SALE20,Armenia
6,2015-03-04,C14975,Cereal,205.7,2844,SALE30,Eritrea
7,2012-05-17,C30645,Clothes,109.3,7299,SALE20,Montenegro
8,2015-01-29,C67225,Vegetables,154.1,2428,SALE10,Jamaica
9,2013-12-24,C98544,Vegetables,154.1,4800,SALE30,Fiji


## 10. Mini-Aggregation — revenue per shipping_city

In [98]:

# 1) Define revenue without tax: price_after_discount * quantity
w = final_data.copy()
w["discount_pct"] = pd.to_numeric(w["discount_pct"], errors="coerce").fillna(0.0)
w["price"] = pd.to_numeric(w["price"], errors="coerce")
w["quantity"] = pd.to_numeric(w["quantity"], errors="coerce")

w["price_after_discount"] = (w["price"] * (1 - w["discount_pct"])).round(2)
w["revenue"] = (w["price_after_discount"] * w["quantity"]).round(2)

# 2) Aggregate by shipping_city (sum of revenue)
city_rev_df = (
    w.groupby("shipping_city", dropna=False)["revenue"]
     .sum()
     .reset_index()
     .sort_values("revenue", ascending=False)
)

# 3) Show top 10 cities
display(city_rev_df.head(10))

# 4) (Optional) Also provide a dict version for the assignment's "map to dict" flavor
city_rev_dict = dict(city_rev_df.itertuples(index=False, name=None))
# Preview only first 5 items to keep output tidy
dict(list(city_rev_dict.items())[:5])

,shipping_city,revenue
120,Papua New Guinea,13461607.93
39,Costa Rica,12202448.45
42,Cuba,11126793.15
122,Prague,10697845.14
154,Tonga,10227070.71
82,Lisbon,9976579.36
145,Swaziland,9910575.72
36,Chad,9060937.83
56,Georgia,9019128.12
86,Luxembourg,8919977.98


{'Papua New Guinea': 13461607.93,
 'Costa Rica': 12202448.450000001,
 'Cuba': 11126793.15,
 'Prague': 10697845.14,
 'Tonga': 10227070.71}

## 11. Serialization Checkpoint

In [99]:
# Select only the required 7 fields for submission
required_cols = ["date","customer_id","product","price","quantity","coupon_code","shipping_city"]
cleaned_data = final_data[required_cols].copy()

# 1) Save as CSV
cleaned_data.to_csv("cleaned_transactions.csv", index=False)

# 2) Save as JSON (records format is usually easiest to reload)
cleaned_data.to_json("cleaned_transactions.json", orient="records", date_format="iso")

print("Step 11: Data serialized to 'cleaned_transactions.csv' and 'cleaned_transactions.json'")

Step 11: Data serialized to 'cleaned_transactions.csv' and 'cleaned_transactions.json'


## 12. Soft Interview Reflection

Using functions in this lab made the workflow more structured and reusable.  
For example, when cleaning data, I could encapsulate the rules in a single `.clean()` method,  
which avoided repeating the same operations across multiple cells.  
Functions also made it easier to test small pieces of logic separately  
(such as parsing coupon codes or computing derived fields) without rewriting code.  

In a real project, this approach improves maintainability:  
if requirements change, I only need to update one function rather than fixing code in many places.  
It also makes collaboration smoother, because teammates can quickly understand  
what each function is supposed to do based on its name and docstring.  
Overall, functions supported cleaner design and faster iteration.

## Data-Dictionary Section

**Sources used**
- **Primary**: `500 Sales Records.csv` (raw transactions)
- **Secondary**: 
  - `Discount_Coupon.csv` → from [Kaggle: Marketing Insights for E-Commerce Company](https://www.kaggle.com/datasets/rishikumarrajvansh/marketing-insights-for-e-commerce-company)  
  - `country_city_lookup.csv` → AI-generated lookup table (synthetic)

**How fields were produced**
- Core transaction columns were **renamed** from the primary dataset.  
- Coupon metadata was **merged by Month** (normalized to Jan–Dec) and missing months filled with defaults.  
- `shipping_city` was derived by mapping `Country → City` via the AI-generated lookup.  
- `customer_id` was **synthetic**, generated from `Order ID`.  
- A simple recency feature (`days_since_purchase`) was added for demonstration of feature engineering.

### Field Definitions

| Field               | Type      | Description                                                                                       | Source      | Creation Notes |
|---------------------|-----------|---------------------------------------------------------------------------------------------------|-------------|----------------|
| `date`              | datetime  | Order date parsed from the primary dataset                                                        | Primary     | Parsed from `Order Date` |
| `customer_id`       | string    | Synthetic customer identifier                                                                     | Derived     | `"C" + (Order ID % 100000)`, zero-padded |
| `product`           | string    | Product / item category                                                                           | Primary     | Renamed from `Item Type` |
| `price`             | float     | Unit price per product                                                                            | Primary     | Renamed from `Unit Price` |
| `quantity`          | int       | Units sold                                                                                       | Primary     | Renamed from `Units Sold` |
| `coupon_code`       | string    | Coupon applied for the month of the order                                                         | Secondary   | From Kaggle `Discount_Coupon.csv`; defaults to `"NONE"` |
| `shipping_city`     | string    | Shipping city resolved from country                                                               | Secondary   | From AI-generated `country_city_lookup.csv`; fallback to `Country` if not matched |
| `discount_pct`      | float     | Coupon discount as a fraction in `[0,1]`                                                          | Secondary   | Normalized from Kaggle coupon dataset; values >1 divided by 100 |
| `Month`             | string    | 3-letter month abbreviation used for coupon join                                                  | Derived     | From `date` (`%b`); helper only |
| `days_since_purchase` | int     | Recency: days since order compared to dataset’s most recent date                                  | Derived     | `max(date) - date`; NaT → NaN |

> **Export policy:** Final submission includes only the **7 required fields**:  
> `date`, `customer_id`, `product`, `price`, `quantity`, `coupon_code`, `shipping_city`.  
> Other columns are shown in the notebook for transparency.
